In [13]:
import pandas as pd
import re
from konlpy.tag import Okt,Hannanum

In [25]:
import konlpy
print(konlpy.__version__)

0.6.0


In [14]:
result_csv = pd.read_csv('result.csv')

- Hannanum은 사용자 사전에 영어 등록이 불가  
- 영어 형태소 분석기로 nltk 사용: React.js 같은 단어가 잘 뽑힘  
-> ```영어는 nltk를 사용하여 분석하고 한글은 Hannanum을 사용해 분석'```

- 7.18: 전체 반복 돌면서 두 가지 형태소 분석기를 사용하여 하나는 영어, 하나는 한글만 뽑아봄
-> 문맥적인 순서 issue 발생
- 7.19: 결과적으로 한국어 형태소 분석기 Okt를 사용하기로 결정

In [15]:
# 형태소 분석 함수 정의
def tokenize(text):
    okt = Okt()
    # 정규표현식을 사용하여 특수문자 제거
    if type(text) is not float:
        text = re.sub(r'[-•·*,\r\n\(\)\[\]\/:ㆍ\<\>＜＞［］]', '', text)
        tokens = okt.morphs(text)  # 형태소만 문자열로 추출
        return tokens

In [16]:
len(result_csv)

11393

In [ ]:
#경력 조건 추가 해야함
columns = ['공고명','자격요건','우대조건','기술 스택','복지']
for column in columns:
    result_csv = result_csv[result_csv[column].apply(lambda x: isinstance(x, str))]
len(result_csv)

result_csv[columns] = result_csv[columns].applymap(tokenize)
result_csv.head(30)

In [18]:
result_csv.to_csv('morp_result_del_ver(1).csv',index = False)

In [19]:
test = pd.read_csv('morp_result.csv')

In [24]:
test= test['복지'].unique()


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

- 자격요건을 통해 기술스택이 nan 값인 부분을 채워야 할까?(보류)  
-> ```추후에 필요하면 추가```
- 기술 스택 종류 파악

In [ ]:
result_csv['기술 스택'] = result_csv['기술 스택'].str.replace('[•·*,\r\n\(\)\[\]\/:ㆍ\<\>＜＞［］]', ' ', regex = True)
word_list = result_csv['기술 스택'].unique()
print(word_list)

word_set = set()

for skill_words in word_list:
    if type(skill_words) is not float:
        english_list = [word.capitalize() for word in skill_words.split() if re.match(r'[ㄱ-ㅎ가-힣]+', word) == None]
        word_set.update(english_list)

# 쓸데없는거 제거해야할 듯
print('기술 스택 종류: ',len(word_set))
print(word_set)
cnt = result_csv[result_csv['기술 스택'].isna() == True]['자격요건'].count()
print('전체 개수: ',cnt)